<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#上下文管理器中的异常捕捉" data-toc-modified-id="上下文管理器中的异常捕捉-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>上下文管理器中的异常捕捉</a></span></li><li><span><a href="#用生成器来定义上下文管理器" data-toc-modified-id="用生成器来定义上下文管理器-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>用生成器来定义上下文管理器</a></span></li></ul></div>

In [2]:
# 上下文管理器是一类定义了 __enter__和__exit__的python 对象
"""
with manager as var:
    do_something(var)
"""
#上面的代码等价于
"""
var = manager.__enter__()
try:
    do_something(var)
finally:
    manager.__exit__()
"""

'\nvar = manager.__enter__()\ntry:\n    do_something(var)\nfinally:\n    manager.__exit__()\n'

In [3]:
class closing(object):
    def __init__(self, obj):
        self.obj = obj
    def __enter__(self):
        return self.obj
    def __exit__(self, *args):
        self.obj.close()

In [4]:
#使用
#with closing(open('/tmp/file', 'w')) as f, 
#实例化了closing类, 并调用__enter__方法, 返回的值传递给f
"""
with closing(open('/tmp/file', 'w')) as f:
  f.write('the content\n')
"""

"\nwith closing(open('/tmp/file', 'w')) as f:\n  f.write('the content\n')\n"

###### 上下文管理器中的异常捕捉

In [5]:
#异常捕捉
#在 with block中出现异常时, 该异常会被传递到__exit__
#__exit__需要接受三个参数: type, value, traceback.
#如果不打算抛出异常, 这三个参数设置成None
#就算抛出了异常, __exit__方法可以通过 return True来吞掉它
#如果__exit__不返回任何值, 那么所有异常都可以被忽视

In [7]:
class assert_raises(object):
    def __init__(self, exp_type):
        self.type = exp_type
    def __enter__(self):
        pass
    def __exit__(self, exp_type, value, traceback):
        if exp_type is None:
            raise AssertionError('exception expected')
        if issubclass(exp_type, self.type):
            #吞掉这个异常
            return True
        raise AssertionError('wrong exception type')
    

In [8]:
with assert_raises(KeyError):
    {}['foo']

###### 用生成器来定义上下文管理器

In [9]:
#这样用
'''
@contextlib.contextmanager
def some_generator(<arguments>):
    <setup>
    try:
        yield <sth>
    finally:
        <clean up>
'''
#使用 contextlib.contextmanager让一个生成器函数变成上下文管理器
#这个生成器函数需要遵循一些规则:
##1. 只 yield 一次。 yield之前的所有代码都被放到 __enter__中去了。而yield之后的部分放到__exit__
##中去了.

'\n@contextlib.contextmanager\ndef some_generator(<arguments>):\n    <setup>\n    try:\n        yield <sth>\n    finally:\n        <clean up>\n'

In [10]:
#使用生成器+contextlib.contextmanager模块使得上下文管理器的定义变得简单
import contextlib
@contextlib.contextmanager
def closing(obj):
    try:
        yield obj
    finally:
        obj.close()

In [ ]:
@contextlib.contextmanager
def assert_raises(exp_type):
    try:
        yield
    except exp_type:
        return
    except Exception as e:
        raise AssertionError('wrong exception type.')
    else:
        raise AssertionError('exception ex')